In [82]:
import random
import os
import numpy as np
import sklearn
import torch
from torch.cuda import manual_seed_all
from torch import manual_seed as torch_manual_seed
from torch.backends import cudnn
import matplotlib as mpl
from matplotlib import pyplot as plt
import torchaudio
import torchaudio.transforms as T

In [83]:
# pre spectrogram augmentations
# these are examples and can be changed based on domain knowledge

def stretch_waveform(waveform, rate=1.2):
    time_stretch = T.TimeStretch()
    # `rate > 1.0` speeds up, `rate < 1.0` slows down
    return time_stretch(waveform, rate)

def shift_pitch(waveform, sample_rate=44100, n_steps = 2):
    pitch_shift = T.PitchShift(sample_rate, n_steps)  # Shift up by 2 semitones
    return pitch_shift(waveform)

def scale_volume(waveform, factor = None):
    if factor is None:
        waveform *= torch.FloatTensor(1).uniform_(0.8, 1.5).item()  # Amplifies waveform by random factor
    else:
        waveform *= factor
    return waveform

def crop_waveform(waveform, crop_size):
    start = torch.randint(0, max(1, waveform.size(-1) - crop_size), (1,)).item()
    return waveform[:, start:start + crop_size]

def apply_reverb(waveform):
    reverb = T.Reverberate()
    return reverb(waveform)

def time_shift(waveform, shift):
    return torch.roll(waveform, shifts=shift, dims=-1)

def add_noise(waveform, noise_level=0.005):
    noise = torch.randn_like(waveform) * noise_level
    return waveform + noise

# Augment on-the-fly stochastically
# again these are just examples and do not necessarily utilize the methods above
def augment_waveform(data):
    waveform, sample_rate = data
    if torch.rand(1).item() > 0.9:
        waveform = add_noise(waveform)
    if torch.rand(1).item() > 0.9:
        waveform = time_shift(waveform, shifts=torch.randint(-waveform.size(-1) // 2, waveform.size(-1) // 2, (1,)).item())
    if torch.rand(1).item() > 0.9:
        waveform = scale_volume(waveform)
    if torch.rand(1).item() > 0.9:
        waveform = apply_reverb(waveform)
    if torch.rand(1).item() > 0.9:
        waveform = shift_pitch(waveform, sample_rate, n_steps= torch.randint(-12, 12, (1,)).item())
    if torch.rand(1).item() > 0.9:
        waveform = stretch_waveform(waveform, rate= torch.FloatTensor(1).uniform_(0.5, 1.5).item())
    return waveform, sample_rate


In [84]:
# Create a MelSpectrogram transformation
mel_spectrogram_transform = T.MelSpectrogram(
    sample_rate=44100,         # Default sample rate, change if needed
    n_fft=1024,                # Number of FFT bins
    hop_length=512,            # Hop length between windows
    n_mels=64                  # Number of Mel bands
)

def waveform_to_spectrogram(data):
    waveform, sample_rate = data
    spectrogram = mel_spectrogram_transform(waveform)  # Apply the spectrogram transformation
    return spectrogram

In [85]:
# post spectrogram augmentations

# Example augmentations, could add more
time_mask = T.TimeMasking(time_mask_param=10)

freq_mask = T.FrequencyMasking(freq_mask_param=8)

# hybridizes two sounds
def mixup(spectrogram1, spectrogram2, alpha=0.2):
    lam = torch.FloatTensor(1).uniform_(0, alpha).item()
    return lam * spectrogram1 + (1 - lam) * spectrogram2

# should probably implement a randomization process like above
def augment_spectrogram(spectrogram):
    augmented = time_mask(spectrogram)  # Apply time masking
    augmented = freq_mask(augmented)   # Apply frequency masking
    return augmented
    

In [86]:
# Decode audio files
def decode_audio(file_tuple):
    file_path, file = file_tuple
    waveform, sample_rate = torchaudio.load(file_path)
    return waveform, sample_rate

In [87]:
import os
import torchaudio
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class UrbanSoundDataset(Dataset):
    def __init__(self, audio_path, fold, csv_path, transform=None):
        self.audio_path = os.path.join(audio_path, f"fold{fold}")
        self.file_list = [os.path.join(self.audio_path, f) for f in os.listdir(self.audio_path) if f.endswith(".wav")]
        self.transform = transform

        # Load the metadata CSV file
        self.metadata = pd.read_csv(csv_path)

    def get_label(self, file_name):
        """Fetch the class label for a given file name from the metadata."""
        label_row = self.metadata.loc[self.metadata['slice_file_name'] == file_name, 'class']
        if not label_row.empty:
            return label_row.values[0]
        else:
            raise ValueError(f"File name {file_name} not found in metadata CSV.")

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        # Load the audio file
        file_path = self.file_list[idx]
        waveform, sample_rate = torchaudio.load(file_path)

        # Convert mono to stereo if necessary
        if waveform.size(0) == 1:  # If mono
            waveform = waveform.repeat(3, 1)

        # Apply transformations
        if self.transform:
            waveform = self.transform(waveform)

        # Extract the file name from the path
        file_name = os.path.basename(file_path)

        # Get the corresponding label for the file
        label = self.get_label(file_name)

        return waveform, label


In [88]:
import torchaudio.transforms as T

# Example transformations
def augment_waveform(waveform):
    # Add your augmentation logic here (e.g., noise addition, time stretch, etc.)
    return waveform

waveform_to_spectrogram = T.MelSpectrogram(sample_rate=16000, n_mels=128)
augment_spectrogram = T.AmplitudeToDB()

# Combine transformations into a callable function
def transform_pipeline(waveform):
    waveform = augment_waveform(waveform)
    spectrogram = waveform_to_spectrogram(waveform)
    # spectrogram = augment_spectrogram(spectrogram)
    return spectrogram

def pad_with_noise(spectrogram, max_time, noise_std=0.01):
    """
    Pads a spectrogram with Gaussian noise instead of zeros.

    Args:
        spectrogram (Tensor): Shape (channels, freq_bins, time_steps)
        max_time (int): Target time dimension
        noise_std (float): Standard deviation of the Gaussian noise

    Returns:
        Tensor: Padded spectrogram with noise
    """
    # Compute how much padding is needed
    pad_amount = max_time - spectrogram.size(2)
    
    if pad_amount > 0:
        # Generate random noise matching the shape of missing time steps
        noise = torch.randn((spectrogram.size(0), spectrogram.size(1), pad_amount)) * noise_std
        
        # Concatenate noise along the time axis
        spectrogram = torch.cat([spectrogram, noise], dim=2)
    
    return spectrogram

# def convert_to_three_channels(spectrogram):
#     # Convert [2, 224, 224] to [3, 224, 224]
#     if spectrogram.size(0) == 2:
#         # Duplicate the first channel to create a third channel
#         return torch.cat((spectrogram, spectrogram[0:1, :, :]), dim=0)
#     return spectrogram

def convert_to_three_channels(spectrogram):
    # Convert [2, 224, 224] to [3, 224, 224]
    if spectrogram.size(0) == 2:
        # Calculate the mean of the two channels
        mean_channel = torch.mean(spectrogram, dim=0, keepdim=True)
        # Concatenate the mean channel as the third channel
        return torch.cat((spectrogram, mean_channel), dim=0)
    return spectrogram

/home/asm2fe/.local/lib/python3.11/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [89]:
import torch
import torchvision
from torch import nn

class densenet(torch.nn.Module):
    """
    DenseNet Class, derived from Pytorch. Intended for model manipulation (i.e. unfreezing layers, etc.)
    To use model, try (densenet).model(data)
    May change to reflect manual implementation of densenet161.
    """
    def __init__(self, weights = "DEFAULT", drop = 0.5):
        super().__init__()  # Initialize the nn.Module base class
        self.model = torchvision.models.densenet161(weights = weights)
        
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(drop),  # Add dropout with 50% probability
            nn.Linear(num_features, 10)  # Adjust for 10 output classes (UrbanSound8k)
        )
        
        # Ensure classifier is trainable
        for param in self.model.classifier.parameters():
            param.requires_grad = True


    def forward(self, x):
        return self.model(x)  # Delegate forward pass to the original DenseNet

    def layer_change(self, layer=0):
        if layer > 0:    
            # Freeze earlier layers (optional)
            for name, param in self.model.features.named_parameters():
                if "conv0" in name or "denseblock1" in name:  # Freeze initial layers and denseblock1
                    param.requires_grad = False
        if layer > 1:    
            # Freeze earlier layers (optional)
            for name, param in self.model.features.named_parameters():
                if "denseblock2" in name:  # Freeze initial layers and denseblock2
                    param.requires_grad = False
        if layer > 2:    
            # Freeze earlier layers (optional)
            for name, param in self.model.features.named_parameters():
                if "denseblock3" in name:  # Freeze initial layers and denseblock3
                    param.requires_grad = False
        if layer > 3:    
            # Freeze earlier layers (optional)
            for name, param in self.model.features.named_parameters():
                if "denseblock4" in name:  # Freeze initial layers and denseblock4
                    param.requires_grad = False

In [90]:
# model = densenet()
# for param in model.model.features.named_parameters():
#     print(param[0])

In [91]:
# Define training and testing loops

def train_loop(train_dataloader, val_dataloader, model, loss_fn, optimizer, scheduler=None, epochs=1):
    model.train()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Store metrics
    epoch_train_losses = []  # Track training loss across epochs
    epoch_val_losses = []  # Track validation loss across epochs
    epoch_val_accuracies = []  # Track validation accuracy across epochs

    # Training loop with early stopping
    early_stopping = EarlyStopping(patience=3, min_delta=0.01)
    
    early_stop_epoch = None
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        size = len(train_dataloader.dataset)
        total_loss = 0  # Initialize variable to accumulate training loss
        

        for batch, (X, y) in enumerate(train_dataloader):
            # Compute prediction and loss
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate loss
            total_loss += loss.item()

            # Print progress periodically
            total_batches = len(train_dataloader)
            if batch % (total_batches // 5) == 0:  # Prints 5 times per epoch
                current = (batch + 1) * len(X)
                print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

        # Average training loss for the epoch
        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Training Loss (Epoch): {avg_train_loss:>7f}")
        epoch_train_losses.append(avg_train_loss)

        # **Validation Step**
        print("Validating...")
        avg_val_loss, val_accuracy = test_loop(val_dataloader, model, loss_fn, verbose=False)
        print(f"Validation Loss: {avg_val_loss:.6f}, Validation Accuracy: {val_accuracy * 100:.2f}%")

        # Track validation metrics
        epoch_val_losses.append(avg_val_loss)
        epoch_val_accuracies.append(val_accuracy)

        if scheduler is not None:
            scheduler.step(val_accuracy)
            print(f"Learning Rate: {scheduler.get_last_lr()}")
        
        # Check for early stopping
        early_stopping(avg_val_loss)
        if early_stopping.stop_training:
            print("Early stopping triggered. Training stopped.")
            early_stop_epoch = epoch
            break

    # Return metrics for tracking/aggregation across folds
    return epoch_train_losses, epoch_val_losses, epoch_val_accuracies, early_stop_epoch

def test_loop(dataloader, model, loss_fn, verbose=True):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Average loss and accuracy for this fold
    avg_test_loss = test_loss / num_batches
    accuracy = correct / size
    if verbose:
        print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {avg_test_loss:>8f} \n")
    return avg_test_loss, accuracy  # Return both average loss and accuracy for this fold

In [92]:
import torchvision.transforms as transforms

def custom_collate_fn(batch):
    # Resize and normalize for DenseNet
    resize_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for DenseNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
    ])
    
    inputs, labels = zip(*batch)  # Separate inputs and labels
    max_time = max(spectrogram.size(2) for spectrogram in inputs)

    # Pad inputs to the same length along the time dimension
    padded_inputs = [
        torch.nn.functional.pad(input, (0, max_time - input.size(2)))
        for input in inputs
    ]

    # Convert to 3 channels and resize
    resized_inputs = [resize_transform(convert_to_three_channels(input)) for input in padded_inputs]
    
    # Map labels to numeric class IDs
    class_mapping = {
        "air_conditioner": 0,
        "car_horn": 1,
        "children_playing": 2,
        "dog_bark": 3,
        "drilling": 4,
        "engine_idling": 5,
        "gun_shot": 6,
        "jackhammer": 7,
        "siren": 8,
        "street_music": 9
    }

    numeric_labels = [class_mapping[label] for label in labels]

    # Stack inputs and labels
    return torch.stack(resized_inputs), torch.tensor(numeric_labels)


In [93]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0):
        """
        Args:
            patience (int): How many epochs to wait for improvement.
            min_delta (float): Minimum change in monitored value to qualify as improvement.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.stop_training = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop_training = True   


In [94]:
print(os.getcwd())

/sfs/gpfs/tardis/home/asm2fe/UrbanAdversary


In [95]:
from sklearn.model_selection import train_test_split
import torch

# Specify paths and batch size
AUDIO_PATH = "./UrbanSound8K/audio"
CSV_PATH = "./UrbanSound8K/metadata/UrbanSound8K.csv"
batch_size = 70
epochs = 20

def setup_seed(seed):
    torch_manual_seed(seed)
    manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    cudnn.deterministic = True

def dense_tune(layer = 0, lr = 2e-4, weight_decay = 0.2, weights = "DEFAULT", drop = 0.5, SEED = 666):
    loss_fn = torch.nn.CrossEntropyLoss()
    setup_seed(SEED)

    # Variables to accumulate metrics across folds
    fold_train_losses = []
    fold_val_losses = []
    fold_val_accuracies = []

    # Loop through folds
    for fold in range(1, 11):
        model = densenet(weights = weights, drop = drop)
        model.layer_change(layer = layer) # freeze first conv and dense block(s) if desired

        print(f"Processing Fold {fold}")
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay = weight_decay)
        # optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay = 0.01, momentum = 0.9)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=0, threshold_mode = 'abs', threshold= 0.01, min_lr=1e-9)

        # Initialize dataset and DataLoader
        dataset = UrbanSoundDataset(audio_path=AUDIO_PATH, fold=fold, transform=transform_pipeline, csv_path=CSV_PATH)
        train_indices, val_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)

        train_dataset = torch.utils.data.Subset(dataset, train_indices)
        val_dataset = torch.utils.data.Subset(dataset, val_indices)

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

        # Train and validate (over multiple epochs per fold)
        epoch_train_losses, epoch_val_losses, epoch_val_accuracies, early_stop_epoch = train_loop(
            train_dataloader, val_dataloader, model, loss_fn, optimizer, scheduler, epochs=epochs
        )

        # Aggregate metrics up to the stopping epoch
        if early_stop_epoch is not None:
            fold_train_losses.append(sum(epoch_train_losses[:early_stop_epoch+1]) / (early_stop_epoch+1))  # Mean up to early stop
            fold_val_losses.append(sum(epoch_val_losses[:early_stop_epoch+1]) / (early_stop_epoch+1))      # Mean up to early stop
            fold_val_accuracies.append(sum(epoch_val_accuracies[:early_stop_epoch+1]) / (early_stop_epoch+1))  # Mean up to early stop
        else:
            # If no early stopping occurred, aggregate metrics across all epochs
            fold_train_losses.append(sum(epoch_train_losses) / len(epoch_train_losses))
            fold_val_losses.append(sum(epoch_val_losses) / len(epoch_val_losses))
            fold_val_accuracies.append(sum(epoch_val_accuracies) / len(epoch_val_accuracies))

        # # Aggregate fold-level metrics (e.g., mean across all epochs)
        # fold_train_losses.append(sum(epoch_train_losses) / len(epoch_train_losses))  # Mean of training losses
        # fold_val_losses.append(sum(epoch_val_losses) / len(epoch_val_losses))        # Mean of validation losses
        # fold_val_accuracies.append(sum(epoch_val_accuracies) / len(epoch_val_accuracies))  # Mean of validation accuracies

    # Compute average metrics across folds
    mean_train_loss = sum(fold_train_losses) / len(fold_train_losses)
    mean_val_loss = sum(fold_val_losses) / len(fold_val_losses)
    mean_val_accuracy = sum(fold_val_accuracies) / len(fold_val_accuracies)

    print(f"\nCross-Validation Results:")
    print(f"Avg Training Loss: {mean_train_loss:.6f}")
    print(f"Avg Validation Loss: {mean_val_loss:.6f}")
    print(f"Avg Validation Accuracy: {mean_val_accuracy * 100:.2f}%")


In [96]:
# dense_tune(layer = 0, lr = 2e-4, weight_decay = 0.2, weights = "DEFAULT", drop = 0.5)

In [97]:

dense_tune(layer = 1, lr = 2e-4, weight_decay = 0.2, weights = "DEFAULT", drop = 0.6)

Processing Fold 1
Epoch 1/20
loss: 2.345630  [   70/  698]
loss: 2.094388  [  210/  698]
loss: 1.921020  [  350/  698]
loss: 1.729051  [  490/  698]
loss: 1.623411  [  630/  698]
Training Loss (Epoch): 1.943488
Validating...
Validation Loss: 1.881623, Validation Accuracy: 43.43%
Learning Rate: [0.0002]
Epoch 2/20
loss: 1.798832  [   70/  698]
loss: 1.305873  [  210/  698]
loss: 1.220195  [  350/  698]
loss: 1.172055  [  490/  698]
loss: 0.929250  [  630/  698]
Training Loss (Epoch): 1.224383
Validating...
Validation Loss: 0.936450, Validation Accuracy: 66.29%
Learning Rate: [0.0002]
Epoch 3/20
loss: 0.772587  [   70/  698]
loss: 0.592083  [  210/  698]
loss: 0.523305  [  350/  698]
loss: 0.455551  [  490/  698]
loss: 0.729686  [  630/  698]
Training Loss (Epoch): 0.664011
Validating...
Validation Loss: 0.852308, Validation Accuracy: 73.71%
Learning Rate: [0.0002]
Epoch 4/20
loss: 0.684272  [   70/  698]
loss: 0.661563  [  210/  698]
loss: 0.340157  [  350/  698]
loss: 0.427867  [  490/

In [98]:

dense_tune(layer = 2, lr = 2e-4, weight_decay = 0.2, weights = "DEFAULT", drop = 0.6)

Processing Fold 1
Epoch 1/20
loss: 2.345630  [   70/  698]
loss: 2.072950  [  210/  698]
loss: 1.931110  [  350/  698]
loss: 1.737942  [  490/  698]
loss: 1.688270  [  630/  698]
Training Loss (Epoch): 1.961481
Validating...
Validation Loss: 1.915651, Validation Accuracy: 45.71%
Learning Rate: [0.0002]
Epoch 2/20
loss: 1.853691  [   70/  698]
loss: 1.421431  [  210/  698]
loss: 1.291020  [  350/  698]


loss: 1.214056  [  490/  698]
loss: 0.979030  [  630/  698]
Training Loss (Epoch): 1.236584
Validating...
Validation Loss: 0.795196, Validation Accuracy: 71.43%
Learning Rate: [0.0002]
Epoch 3/20
loss: 0.723062  [   70/  698]
loss: 0.591281  [  210/  698]
loss: 0.678622  [  350/  698]
loss: 0.348995  [  490/  698]
loss: 0.502151  [  630/  698]
Training Loss (Epoch): 0.601294
Validating...
Validation Loss: 0.526140, Validation Accuracy: 80.00%
Learning Rate: [0.0002]
Epoch 4/20
loss: 0.420100  [   70/  698]
loss: 0.451533  [  210/  698]
loss: 0.253627  [  350/  698]
loss: 0.374431  [  490/  698]
loss: 0.278992  [  630/  698]
Training Loss (Epoch): 0.320396
Validating...
Validation Loss: 0.447303, Validation Accuracy: 84.00%
Learning Rate: [0.0002]
Epoch 5/20
loss: 0.195092  [   70/  698]
loss: 0.275549  [  210/  698]
loss: 0.486779  [  350/  698]
loss: 0.291594  [  490/  698]
loss: 0.569473  [  630/  698]
Training Loss (Epoch): 0.286737
Validating...
Validation Loss: 0.508612, Validatio

In [99]:

dense_tune(layer = 3, lr = 2e-4, weight_decay = 0.2, weights = "DEFAULT", drop = 0.6)

Processing Fold 1
Epoch 1/20
loss: 2.345630  [   70/  698]
loss: 2.072877  [  210/  698]
loss: 1.962428  [  350/  698]
loss: 1.739533  [  490/  698]
loss: 1.749560  [  630/  698]
Training Loss (Epoch): 1.983971
Validating...
Validation Loss: 1.925203, Validation Accuracy: 46.29%
Learning Rate: [0.0002]
Epoch 2/20
loss: 1.898814  [   70/  698]
loss: 1.599640  [  210/  698]
loss: 1.364550  [  350/  698]
loss: 1.153509  [  490/  698]
loss: 1.062906  [  630/  698]
Training Loss (Epoch): 1.334219
Validating...
Validation Loss: 0.926143, Validation Accuracy: 68.57%
Learning Rate: [0.0002]
Epoch 3/20
loss: 0.839380  [   70/  698]
loss: 0.883843  [  210/  698]
loss: 0.554768  [  350/  698]
loss: 0.439027  [  490/  698]
loss: 0.636104  [  630/  698]
Training Loss (Epoch): 0.677705
Validating...
Validation Loss: 0.638317, Validation Accuracy: 74.29%
Learning Rate: [0.0002]
Epoch 4/20
loss: 0.438796  [   70/  698]
loss: 0.381360  [  210/  698]
loss: 0.357722  [  350/  698]
loss: 0.269120  [  490/

In [100]:

dense_tune(layer = 4, lr = 2e-4, weight_decay = 0.2, weights = "DEFAULT", drop = 0.6)

Processing Fold 1
Epoch 1/20
loss: 2.345630  [   70/  698]
loss: 2.251592  [  210/  698]
loss: 2.221283  [  350/  698]
loss: 2.066443  [  490/  698]
loss: 2.048594  [  630/  698]
Training Loss (Epoch): 2.198322
Validating...
Validation Loss: 2.172462, Validation Accuracy: 33.71%
Learning Rate: [0.0002]
Epoch 2/20
loss: 2.187246  [   70/  698]
loss: 2.087860  [  210/  698]
loss: 1.968873  [  350/  698]
loss: 1.856637  [  490/  698]
loss: 1.765840  [  630/  698]
Training Loss (Epoch): 1.928014
Validating...
Validation Loss: 1.618063, Validation Accuracy: 50.29%
Learning Rate: [0.0002]
Epoch 3/20
loss: 1.639433  [   70/  698]
loss: 1.311768  [  210/  698]
loss: 1.326423  [  350/  698]
loss: 1.108037  [  490/  698]
loss: 1.128855  [  630/  698]
Training Loss (Epoch): 1.283857
Validating...
Validation Loss: 1.093366, Validation Accuracy: 60.57%
Learning Rate: [0.0002]
Epoch 4/20
loss: 1.027994  [   70/  698]
loss: 0.938204  [  210/  698]
loss: 0.800702  [  350/  698]
loss: 0.747084  [  490/

In [101]:
dense_tune(layer = 0, lr = 2e-3, weight_decay = 0.2, weights = "DEFAULT", drop = 0.6)

Processing Fold 1
Epoch 1/20
loss: 2.345630  [   70/  698]
loss: 2.274077  [  210/  698]
loss: 2.140732  [  350/  698]
loss: 2.044379  [  490/  698]
loss: 2.015721  [  630/  698]
Training Loss (Epoch): 2.188375
Validating...
Validation Loss: 327.955607, Validation Accuracy: 10.86%
Learning Rate: [0.002]
Epoch 2/20
loss: 306.491974  [   70/  698]
loss: 2.882579  [  210/  698]
loss: 2.343071  [  350/  698]
loss: 2.359900  [  490/  698]
loss: 2.283572  [  630/  698]
Training Loss (Epoch): 33.465685
Validating...
Validation Loss: 2.239516, Validation Accuracy: 11.43%
Learning Rate: [0.001]
Epoch 3/20
loss: 2.258096  [   70/  698]
loss: 2.193193  [  210/  698]
loss: 2.309035  [  350/  698]
loss: 2.340590  [  490/  698]
loss: 2.252954  [  630/  698]
Training Loss (Epoch): 2.265103
Validating...
Validation Loss: 2.230498, Validation Accuracy: 14.29%
Learning Rate: [0.001]
Epoch 4/20
loss: 2.256372  [   70/  698]
loss: 2.275185  [  210/  698]
loss: 2.336182  [  350/  698]
loss: 2.257104  [  49

KeyboardInterrupt: 

In [ ]:
# import torchaudio

# audio_path = "./UrbanSound8k/audio/fold1/137156-9-0-30.wav"
# waveform, sample_rate = torchaudio.load(audio_path)
# print(f"Shape: {waveform.shape}, Sample Rate: {sample_rate}")

In [ ]:
def show_stereogram(spectrogram):
    # Convert to numpy
    spectrogram_np = spectrogram.numpy()  # Shape: (2, Freq, Time)

    # Plot left and right channels
    fig, axs = plt.subplots(2, 1, figsize=(6, 6), constrained_layout=True)

    axs[0].imshow(spectrogram_np[0], aspect='auto', origin='lower', cmap='magma')
    axs[0].set_title(f"Spectrogram {i+1} - Left Channel")
    axs[0].set_ylabel("Frequency Bins")
    axs[0].set_xlabel("Time Frames")

    axs[1].imshow(spectrogram_np[1], aspect='auto', origin='lower', cmap='magma')
    axs[1].set_title(f"Spectrogram {i+1} - Right Channel")
    axs[1].set_ylabel("Frequency Bins")
    axs[1].set_xlabel("Time Frames")

    plt.show()